In [74]:
import math
import numpy as np
from numba import cuda, jit, int32
from datetime import datetime

In [94]:
TPB = 32
len_ = 1000
a = []
for i in range(len_):
  a.append(i)
b= []
for i in range(int((len_)/2)):
  b.append(a[-1])
  a.pop()

col_ = (math.ceil((len(a))/TPB))

c1 = np.zeros(1, dtype = np.int64)
start = datetime.now()
Z = sum(a) + sum(b)
print(Z)
Z_time = datetime.now()-start
print(Z_time)

499500
0:00:00.000311


In [82]:
@cuda.jit
def dev_vec_sum(a, b, c):
  idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  sum = 0
  if idx < len(a):
    
    sum += a[idx]+b[idx]
    cuda.atomic.add(c, 0, sum)
    
def DeviceHost(a, b, c, col_):
    dev_a = cuda.to_device(a)
    dev_b = cuda.to_device(b)
    dev_c = cuda.to_device(c)
    start = datetime.now()
    dev_vec_sum[col_, TPB](dev_a, dev_b, dev_c)
    itog_time = datetime.now()-start
    print(datetime.now()-start)
    return dev_c.copy_to_host(), itog_time

In [83]:
res, itog_time = DeviceHost(a, b, c1, col_)
print(Z, Z_time)
print(res[0], itog_time)

0:00:00.174547
499500 0:00:00.000214
499500 0:00:00.174539


In [106]:
itog_mass = []
Z_mass = []
TPB = 32
len_ = 1000
for k_ in range(5): 
  
  a = []
  for i in range(len_):
    a.append(i)
  b= []
  for i in range(int((len_)/2)):
    b.append(a[-1])
    a.pop()
  col_ = (math.ceil((len(a))/TPB))  
  c1 = np.zeros(1, dtype = np.int64)
  start = datetime.now()
  Z = sum(a) + sum(b)
  Z_time = datetime.now()-start

  res, itog_time = DeviceHost(a, b, c1, col_)
  print(Z, Z_time)
  print(res[0], itog_time)
  itog_mass.append(itog_time)
  Z_mass.append(Z_time)
  len_ = len_*10

0:00:00.000542
499500 0:00:00.000019
499500 0:00:00.000537
0:00:00.000691
49995000 0:00:00.000069
49995000 0:00:00.000684
0:00:00.000370
4999950000 0:00:00.000751
4999950000 0:00:00.000365
0:00:00.000563
499999500000 0:00:00.007951
499999500000 0:00:00.000556
0:00:00.000521
49999995000000 0:00:00.073925
49999995000000 0:00:00.000516


In [104]:
print(Z_mass)
print(itog_mass)

[datetime.timedelta(microseconds=11), datetime.timedelta(microseconds=109), datetime.timedelta(microseconds=788), datetime.timedelta(microseconds=7090), datetime.timedelta(microseconds=74538)]
[datetime.timedelta(microseconds=522), datetime.timedelta(microseconds=422), datetime.timedelta(microseconds=756), datetime.timedelta(microseconds=518), datetime.timedelta(microseconds=496)]
